In [ ]:
import csv
from mysql.connector import connect, Error
import re
import datetime


class MySQLDatabase:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.connection = None

    def connect(self):
        try:
            self.connection = connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            print("Connected to MySQL database")
        except Error as e:
            print(f"Error connecting to MySQL database: {e}")

    def close_connection(self):
        if self.connection and self.connection.is_connected():
            self.connection.close()
            print("MySQL connection closed.")

    def create_table(self):
        try:
            cursor = self.connection.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS employees (
                    Name VARCHAR(60),
                    Age INT(50),
                    Address VARCHAR(255),
                    Mobile_Number VARCHAR(255),
                    Gender VARCHAR(50),
                    Education_Details VARCHAR(50),
                    Salary INT(50),
                    DOJ DATE,
                    Department VARCHAR(255),
                    Position VARCHAR(255)
                )
            """)
            print("Table created successfully")
        except Error as e:
            print(f"Error creating table in MySQL database: {e}")

    def insert_data_from_csv(self, csv_file_path):
        try:
            cursor = self.connection.cursor()
            with open(csv_file_path, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header row
                records_inserted = 0
                for row in reader:
                    cursor.execute("""
                        INSERT INTO employees (Name, Age, Address, Mobile_Number, Gender, Education_Details, Salary, DOJ, Department, Position)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, row)
                    records_inserted += 1
            self.connection.commit()
            print(f"{records_inserted} records inserted successfully")
        except Error as e:
            print(f"Error inserting data into MySQL table: {e}")

    # Other methods for validation, updating, deleting, listing, etc.
# Function to validate name input
def validate_name(name):
   if not name.replace(" ", "").isalpha():
       print("Give me a valid name.")
       return False
   elif len(name) > 20:
       print("Name length should not exceed 20 characters.")
       return False
   else:
       return True

# Function to validate age input
def validate_age(age):
   if not age.isdigit():
       print("Give me a valid age.")
       return False
   else:
       return True

# Function to validate address input
def validate_address(address):
   num_count = sum(c.isdigit() for c in address)
   if num_count < 1 or num_count > 2:
       print("Address should start with one or two numbers.")
       return False
   elif not address[num_count:].isalpha():
       print("Address should only contain letters after the initial numbers.")
       return False
   else:
       return True

# Function to validate mobile number input
def validate_mobile_number(mobile_number):
   if not mobile_number.startswith("+91"):
       print("Mobile number should start with +91.")
       return False
   elif len(mobile_number) != 13 or not mobile_number[3:].isdigit():
       print("Mobile number should contain only digits after +91.")
       return False
   elif not set(mobile_number[3:4]) <= set("6789"):
       print("Mobile number should contain only one digit from 8, 7, 6, 9 after +91.")
       return False
   else:
       return True

# Function to validate gender input
def validate_gender(gender):
   gender = gender.strip()  # Remove leading and trailing whitespace
   if len(gender) != 1 or not gender.isalpha() or gender.upper() not in ["M", "F"]:
       print("Gender should be either 'M' or 'F', without spaces, digits, or special characters.")
       return False
   else:
       return True

# Function to validate education details input
def validate_education_details(education):
   if not education.replace(" ", "").isalpha():
       print("Education details should only contain alphabets.")
       return False
   else:
       return True

# Function to validate salary input
def validate_salary(salary):
   # Regular expression pattern to match digits only
   pattern = r'^\d+$'
   # Check if the salary matches the pattern
   if re.match(pattern, salary):
       return True
   else:
       print("Salary should contain only digits.")
       return False

# Function to validate date of joining input
def validate_date_of_joining(doj):
   # Regular expression pattern to match YYYY-MM-DD format
   pattern = r'^\d{4}-\d{2}-\d{2}$'

   # Check if the date of joining matches the pattern
   if not re.match(pattern, doj):
       print("Date of joining should follow the format YYYY-MM-DD.")
       return False

   # Extract year, month, and day from the date of joining
   year, month, day = map(int, doj.split('-'))

   # Get the current date
   current_date = datetime.datetime.now().date()

   # Check if the year is not greater than the current year
   if year > current_date.year:
       print("Year should not be greater than the current year.")
       return False

   # Check if the month is not greater than 12
   if month > 12:
       print("Month should not be greater than 12.")
       return False


   # Check if the day is not greater than 31
   if day > 31:
       print("Day should not be greater than 31.")
       return False


   return True


# Function to validate department input
def validate_department(department):
   if not department.replace(" ", "").isalpha():
       print("Department should only contain alphabets.")
       return False
   else:
       return True


# Function to validate position input
def validate_position(position):
   if not position.replace(" ", "").isalpha():
       print("Position should only contain alphabets.")
       return False
   else:
       return True


# Function to add a new employee
def add_new_employee(connection):
   cursor = connection.cursor()

   # Get employee details from user input
   name = input("Enter name: ")
   if not validate_name(name):
       return

   age = input("Enter age: ")
   if not validate_age(age):
       return

   address = input("Enter address: ")
   if not validate_address(address):
       return


   mobile_number = input("Enter mobile number: ")
   if not validate_mobile_number(mobile_number):
       return


   gender = input("Enter gender: ")
   if not validate_gender(gender):
       return


   education_details = input("Enter education details: ")
   if not validate_education_details(education_details):
       return


   salary = input("Enter salary: ")
   if not validate_salary(salary):
       return


   doj = input("Enter date of joining (YYYY-MM-DD): ")
   if not validate_date_of_joining(doj):
       return


   department = input("Enter department: ")
   if not validate_department(department):
       return


   position = input("Enter position: ")
   if not validate_position(position):
       return


   # Insert new employee data into MySQL table
   try:
       cursor.execute("""
           INSERT INTO employees (Name, Age, Address, Mobile_Number, Gender, Education_Details, Salary, DOJ, Department, Position)
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
       """, (name, age, address, mobile_number, gender, education_details, salary, doj, department, position))
       connection.commit()
       print("New employee added successfully")
   except Error as e:
       print(f"Error adding new employee: {e}")
   finally:
       if connection.is_connected():
           cursor.close()


# Function to view a particular employee's details
def view_employee_details(connection):
   cursor = connection.cursor()


   # Get the name of the employee whose details to view
   employee_name = input("Enter the name of the employee: ")


   # Query the database to retrieve the details of the employee
   try:
       cursor.execute("""
           SELECT * FROM employees WHERE Name = %s
       """, (employee_name,))


       # Fetch all results
       employee_details = cursor.fetchall()


       if employee_details:
           for idx, employee in enumerate(employee_details, start=1):
               print(f"\nEmployee Details {idx}:")
               print("Name:", employee[0])
               print("Age:", employee[1])
               print("Address:", employee[2])
               print("Mobile Number:", employee[3])
               print("Gender:", employee[4])
               print("Education Details:", employee[5])
               print("Salary:", employee[6])
               print("Date of Joining:", employee[7])
               print("Department:", employee[8])
               print("Position:", employee[9])
       else:
           print("Employee not found.")


   except Error as e:
       print(f"Error retrieving employee details: {e}")


   finally:
           cursor.close()


## Function to update employee information (excluding name)
def update_employee(connection):
   cursor = connection.cursor()
   # Get the name of the employee whose details to update
   employee_name = input("Enter the name of the employee to update: ")


   try:
       # Check if the employee exists in the database
       cursor.execute("SELECT * FROM employees WHERE Name = %s", (employee_name,))
       employee = cursor.fetchone()


       if employee:
           # Display current details of the employee
           print("\nCurrent Employee Details:")
           print("Name:", employee[0])
           print("Age:", employee[1])
           print("Address:", employee[2])
           print("Mobile Number:", employee[3])
           print("Gender:", employee[4])
           print("Education Details:", employee[5])
           print("Salary:", employee[6])
           print("Date of Joining:", employee[7])
           print("Department:", employee[8])
           print("Position:", employee[9])


           # Consume any unread result before executing another statement
           cursor.fetchall()


           # Get updated information from user input
           age = input("Enter new age: ")
           address = input("Enter new address: ")
           mobile_number = input("Enter new mobile number: ")
           gender = input("Enter new gender: ")
           education_details = input("Enter new education details: ")
           salary = input("Enter new salary: ")
           doj = input("Enter new date of joining (YYYY-MM-DD): ")
           department = input("Enter new department: ")
           position = input("Enter new position: ")


           # Update employee information in the database
           cursor.execute("""
               UPDATE employees
               SET Age = %s, Address = %s, Mobile_Number = %s, Gender = %s,
                   Education_Details = %s, Salary = %s, DOJ = %s, Department = %s, Position = %s
               WHERE Name = %s
           """, (age, address, mobile_number, gender, education_details,
                 salary, doj, department, position, employee_name))
           connection.commit()
           print("Employee information updated successfully.")
       else:
           print("Employee not found.")


   except Error as e:
       print(f"Error updating employee information: {e}")


   finally:
       if connection.is_connected():
           cursor.close()


# Function to delete employee records who worked less than 1 month
def delete_inactive_employees(connection):
   cursor = connection.cursor()


   try:
       # Calculate today's date
       current_date = datetime.date.today()


       # Calculate the date 1 month ago
       one_month_ago = current_date - datetime.timedelta(days=30)


       # Delete employee records where DOJ is within the last month
       cursor.execute("DELETE FROM employees WHERE DOJ >= %s AND DOJ <= %s", (one_month_ago, current_date))
       connection.commit()
       print("Inactive employee records deleted successfully")


   except Error as e:
       print(f"Error deleting employee records: {e}")


   finally:
       if connection.is_connected():
           cursor.close()




# Function to list all employees in the organization with department, position, gender, and employee name
           # Function to list all employees in the organization
def list_all_employees(connection):
               cursor = connection.cursor()


               try:
                   # Query the database to retrieve all employees' details
                   cursor.execute("SELECT Distinct Name, Department, Position, Gender FROM employees")


                   # Fetch all results
                   all_employees = cursor.fetchall()


                   if all_employees:
                       print("\nAll Employees in the Organization:")
                       for idx, employee in enumerate(all_employees, start=1):
                           print(f"\nEmployee {idx}:")
                           print("Name:", employee[0])
                           print("Department:", employee[1])
                           print("Position:", employee[2])
                           print("Gender:", employee[3])
                   else:
                       print("No employees found in the organization.")


               except Error as e:
                   print(f"Error listing all employees: {e}")


               finally:
                   if connection.is_connected():
                       cursor.close()
# Function to calculate total monthly salary for each employee
def calculate_total_salary(connection):
   try:
       cursor = connection.cursor()


       # SQL query to calculate total monthly salary for each employee
       query = """
           SELECT Name, SUM(Salary) AS TotalSalary
           FROM employees
           GROUP BY Name
       """


       cursor.execute(query)
       result = cursor.fetchall()


       if result:
           print("\nTotal Monthly Salary for Each Employee:")
           for idx, row in enumerate(result, start=1):
               print(f"\nEmployee {idx}:")
               print("Name:", row[0])
               print("Total Salary:", row[1])
       else:
           print("No employee records found.")

       return result

   except Error as e:
       print(f"Error calculating total salary: {e}")

   finally:
       if connection.is_connected():
           cursor.close()

# Function to export employee data to a CSV file
def export_employee_data_to_csv(connection, csv_file_path):
    try:
        cursor = connection.cursor()

        # SQL query to select all employee data
        query = """
            SELECT Name, Age, Address, Mobile_Number, Gender, Education_Details, Salary, DOJ, Department, Position
            FROM employees
        """

        # Execute the query 
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Write data to CSV file
        with open('/home/nineleaps/Downloads/exports.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            # Write header row
            writer.writerow(['Name', 'Age', 'Address', 'Mobile_Number', 'Gender', 'Education_Details', 'Salary', 'DOJ', 'Department', 'Position'])
            # Write data rows
            writer.writerows(results)

        print(f"Employee data exported to '{csv_file_path}' successfully.")

    except Error as e:
        print(f"Error exporting employee data to CSV: {e}")

    finally:
        if connection.is_connected():
            cursor.close()

# Function to delete duplicate entries based on 'Name' column
def delete_duplicate_names(connection):
    try:
        cursor = connection.cursor()

        # SQL query to delete duplicate entries based on 'Name' column
        query = """
            DELETE t1 FROM employees t1
            INNER JOIN employees t2
            WHERE t1.Name = t2.Name AND t1.rowid < t2.rowid
        """

        cursor.execute(query)
        connection.commit()
        print("Duplicate entries deleted successfully")

    except Error as e:
        print(f"Error deleting duplicate entries: {e}")

    finally:
        if connection.is_connected():
            cursor.close()

def main():
    db = MySQLDatabase("127.0.0.1", "root", "Nayan@12345", "ems3project")
    db.connect()
    db.create_table()
    db.insert_data_from_csv('/home/nineleaps/Downloads/final.csv')

    while True:
        print("\nMenu:")
        print("1. Add a new employee")
        print("2. View a particular employee's details")
        print("3. Update employee information")
        print("4. Delete employee record (soft delete)")
        print("5. List all employees in the organization")
        print("6. Calculate total monthly salary for each employee")
        print("7. Export employee data to a CSV file")
        print("8. Exit")

        option = input("Enter your choice (1, 2, 3, 4, 5, 6, 7, 8): ")

        if option == '1':
            add_new_employee(db.connection)
        elif option == '2':
            view_employee_details(db.connection)
        elif option == '3':
            update_employee(db.connection)
        elif option == '4':
            delete_inactive_employees(db.connection)
        elif option == '5':
            list_all_employees(db.connection)
        elif option == '6':
            calculate_total_salary(db.connection)
        elif option == '7':
            export_employee_data_to_csv(db.connection, '/home/nineleaps/Downloads/exports.csv')
        elif option == '8':
            break
        else:
            print("Invalid option. Please try again.")

    db.close_connection()


if __name__ == "__main__":
    main()

Connected to MySQL database
Table created successfully
101 records inserted successfully

Menu:
1. Add a new employee
2. View a particular employee's details
3. Update employee information
4. Delete employee record (soft delete)
5. List all employees in the organization
6. Calculate total monthly salary for each employee
7. Export employee data to a CSV file
8. Exit


Enter your choice (1, 2, 3, 4, 5, 6, 7, 8):  7


Employee data exported to '/home/nineleaps/Downloads/exports.csv' successfully.

Menu:
1. Add a new employee
2. View a particular employee's details
3. Update employee information
4. Delete employee record (soft delete)
5. List all employees in the organization
6. Calculate total monthly salary for each employee
7. Export employee data to a CSV file
8. Exit
